# Predict characters using 1 cell RNN using Chainer

### Basics

In [85]:
import numpy as np
import sys
import glob
import shutil

### Create the dataset

In [86]:
def create_dataset(ds_path):
    data = open(ds_path, 'r').read()
    symbols = list(set(data))
    char_idx = {char:idx for idx, char in enumerate(symbols)}
    idx_char = {idx:char for idx, char in enumerate(symbols)}
    idx = [char_idx[char] for idx, char in enumerate(data)]
    return char_idx, idx_char, np.asarray(idx, dtype=np.int32), data

### RNN Model

In [108]:
import chainer
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

In [134]:
class RNN(chainer.Chain):
    def __init__(self, n_vocab, n_units):
        super(RNN, self).__init__(
            embedID = L.EmbedID(n_vocab, n_units),
            l1 = L.Linear(n_units, n_units),
            h1 = L.Linear(n_units, n_units),
            l2 = L.Linear(n_units, n_vocab)
        )
        self.r_h1 = None
        
    def reset_state(self):
        self.r_h1 = None
    
    def __call__(self, x):
        h = self.embedID(x)
        if self.r_h1 is None:
            self.r_h1 = F.tanh(self.l1(h))
        else:
            self.r_h1 = F.tanh(self.l1(h) + self.h1(self.r_h1))
        y = self.l2(self.r_h1)
        return y
            

### Setup the dataset iterator

In [163]:
class ParallelSequentialIterator(chainer.dataset.Iterator):
 
    def __init__(self, dataset, batch_size, repeat=True):
        self.dataset = dataset
        self.batch_size = batch_size
        self.epoch = 0
        self.is_new_epoch = False
        self.repeat = repeat
        length = len(dataset)
        self.offsets = [i * length // batch_size for i in range(batch_size)]
        self.iteration = 0
 
    def __next__(self):
        length = len(self.dataset)
        if not self.repeat and self.iteration * self.batch_size >= length:
            raise StopIteration
        cur_words = self.get_words()
        self.iteration += 1
        next_words = self.get_words()
 
        epoch = self.iteration * self.batch_size // length
        self.is_new_epoch = self.epoch < epoch
        if self.is_new_epoch:
            self.epoch = epoch
         
        return list(zip(cur_words, next_words))
 
    @property
    def epoch_detail(self):
        return self.iteration * self.batch_size / len(self.dataset)
 
    def get_words(self):
        return [self.dataset[(offset + self.iteration) % len(self.dataset)]
                for offset in self.offsets]
 
    def serialize(self, serializer):
        self.iteration = serializer('iteration', self.iteration)
        self.epoch = serializer('epoch', self.epoch)

### Setup the updater

In [164]:
class BPTTUpdater(chainer.training.StandardUpdater):
 
    def __init__(self, train_iter, optimizer, bprop_len, device):
        super(BPTTUpdater, self).__init__(
            train_iter, optimizer, device=device)
        self.bprop_len = bprop_len
 
    def update_core(self):
        loss = 0
        train_iter = self.get_iterator('main')
        optimizer = self.get_optimizer('main')
 
        for i in range(self.bprop_len):
            batch = train_iter.__next__()
            x, t = self.converter(batch, self.device)
            loss += optimizer.target(chainer.Variable(x), chainer.Variable(t))
        optimizer.target.cleargrads()
        loss.backward()
        loss.unchain_backward()
        optimizer.update()

### Setup the training script

In [172]:
n_epochs = 5  # epochs
n_units = 100   # neurons in the hidden cell
batch_size = 1
train_test_split = 2

In [ ]:
char_idx, idx_char, idx, train_data = create_dataset('dataset/fewlines.txt')
model = RNN(len(char_idx), n_units)
model = L.Classifier(model)

eval_classifier_model = model.copy()
eval_model = model.predictor

optimizer = optimizers.Adam(alpha = 0.0005)
optimizer.setup(model)

train = idx[0:len(idx) - train_test_split] #len(idx) = 4583798
test = idx[len(idx) - train_test_split : -1]

train_iter = ParallelSequentialIterator(train, batch_size)
test_iter = ParallelSequentialIterator(test, batch_size, repeat=False)

updater = BPTTUpdater(train_iter, optimizer, 10, -1)

trainer = chainer.training.Trainer(updater, (n_epochs, 'epoch'), out='results')

trainer.extend(extensions.Evaluator(test_iter, eval_classifier_model, device = -1,eval_hook=lambda _: eval_model.reset_state()))

trainer.extend(extensions.dump_graph('main/loss'))

interval = (1, 'epoch')

trainer.extend(extensions.snapshot_object(model, 'epoch-{.updater.epoch}.model'), trigger=interval)
trainer.extend(extensions.snapshot(), trigger=interval)
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], x_key='epoch', file_name='accuracy.png'))

trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           3.65307     2.94902               0.16724        0.27572                   61.1466       
2           2.66957     2.51293               0.30996        0.32918                   122.853       


### Prediction

In [171]:
model = RNN(len(char_idx), n_units)

model = L.Classifier(model)
serializers.load_npz('results/epoch-3.model', model)

data = np.asarray([2], dtype=np.int32)

for i in range(0, 100):
    
    pred_model = model.predictor(Variable(data))
    predicted_char = idx_char[np.argmax(pred_model.data)]
    print(predicted_char, end='')
    data = np.asarray([char_idx[predicted_char]],dtype=np.int32)

here are you."
"Where are you."
"Where are you."
"Where are you."
"Where are you."
"Where are you."
